In [6]:
from __future__ import division
import numpy as np
import scipy as sp
import pandas as pd
import csv
import random as rn

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [7]:
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import KernelPCA, PCA
from sklearn.linear_model import LogisticRegressionCV
from sklearn import svm, cross_validation, grid_search
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import f_classif, RFECV, SelectKBest

In [8]:
ls ../datasets/KDD-Cup

Newcup98LRN.txt  cup98LRN.txt


In [9]:
cup_csv = "../datasets/KDD-Cup/Newcup98LRN.txt"
fi = open(cup_csv, 'rb')
data = fi.read()
fi.close()
fo = open('../datasets/KDD-Cup/Newcup98LRN.txt', 'wb')
fo.write(data.replace('\x00', ''))
fo.close()

In [15]:
fields = []

with open(cup_csv, 'rb') as ccsv:
    reader = csv.reader(ccsv)
    for row in reader:
        fields.append(row)

In [16]:
feat_df = pd.DataFrame(fields[1:], columns=fields[0])

In [17]:
feat_df = feat_df.replace({' ': np.nan})
feat_df = feat_df.replace({'': np.nan})
feat_df = feat_df.dropna(axis=1)

In [18]:
feat_df.head()

,ODATEDW,TCODE,STATE,ZIP,DOB,MDMAUD,HIT,MALEMILI,MALEVET,VIETVETS,...,CONTROLN,TARGET_B,TARGET_D,HPHONE_D,RFA_2R,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A
0,8901,0,IL,61081,3712,XXXX,0,0,39,34,...,95515,0,0,0,L,4,E,X,X,X
1,9401,1,CA,91326,5202,XXXX,16,0,15,55,...,148535,0,0,0,L,2,G,X,X,X
2,9001,1,NC,27017,0,XXXX,2,0,20,29,...,15078,0,0,1,L,4,E,X,X,X
3,8701,0,CA,95953,2801,XXXX,2,0,23,14,...,172556,0,0,1,L,4,E,X,X,X
4,8601,0,FL,33176,2001,XXXX,60,1,28,9,...,7112,0,0,1,L,2,F,X,X,X


In [19]:
feat_df.drop("TARGET_D", axis=1, inplace=True)

In [ ]:
indices = feat_df.columns
feat_df = feat_df[:20000]

In [ ]:
attrs = []

for col in indices:
    attr_vals = set(feat_df[col])
    attrs.append(list(attr_vals))
    val_map = {val:i for val, i in zip(attrs[-1], range(len(attr_vals)))}
    feat_df[col] = feat_df[col].replace(val_map)

In [ ]:
# 1-hot encoding
for attr, old_col_name in zip(attrs, indices):
    if len(attr) > 2 and len(attr) < 31:
        for label, col_name in zip(range(len(attr)), attr):
            feat_df[str(col_name)+old_col_name] = feat_df[[old_col_name]].applymap(lambda x: 1 if x == label else 0)

        feat_df.drop(old_col_name, axis=1, inplace=True)

In [ ]:
feat_df.head()


In [ ]:
y = feat_df.TARGET_B.as_matrix()
feat_df.drop("TARGET_B", axis=1, inplace=True)

X = feat_df.as_matrix()

In [ ]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(
 X,y, train_size=5000, random_state=20)

# KDD-Cup

In [ ]:
# Logit
logit_clf = LogisticRegressionCV(solver='liblinear', Cs=[.001,.01,.1,1,10,100], penalty='l1')
logit_clf.fit(X_train, y_train)
logit_clf.score(X_test, y_test)

In [ ]:
# SVM 
svm_params = {'C':[.01,1,10,100,500], 'degree':[1,2,3]}
clf = svm.SVC(kernel='poly')
svm_clf = grid_search.GridSearchCV(clf, svm_params, cv=3, n_jobs=4)
svm_clf.fit(X_train,y_train)
print svm_clf.best_score_
svm_clf.best_params_

In [ ]:
# Random Forests
rf_params = {"max_features":[1,2,4,6,8,12,16,20], 
             "n_estimators":[1024]}
clf = RandomForestClassifier()
rf_clf = grid_search.GridSearchCV(clf, rf_params, cv=3, n_jobs=4)
rf_clf.fit(X_train,y_train)
print rf_clf.best_score_
rf_clf.best_params_

In [ ]:
# KNN
knn_params = {"n_neighbors":[1,2,3,5,7,10,15,25,50,100,500]}
clf = KNeighborsClassifier()
knn_clf = grid_search.GridSearchCV(clf, knn_params, cv=3, n_jobs=4)
knn_clf.fit(X_train,y_train);
print knn_clf.best_score_
knn_clf.best_params_